### 3d visualization libraries

 * VisPy (http://vispy.org) install via sudo pip install vispy
 * plotly (http://plot.ly/) available with pip install plotly
 
As mentioned before *plotly* is a high-level library whose primary purpose is getting your graphics on-line as quickly and seamlessly as possible. 
 
Vispy on the other hand is a library whose primary purpose ensuring all the most computationally-intensive part of your graphics rendering happens on your GPU (the CPU of your graphics card).  This library is meant for building software with *fast* interactive graphics, often using *large* data sets.   Vispy is developing quickly, so beware many examples may not run without the latest version installed. 

Vispy is basically an object-oriented front-end for something called the OpenGL standard.  This is a language for interacting with graphics processors that is agreed upon by large hardware and software manufacturers. 

In [ ]:
## gives current vispy version -- ideally you will have 0.5.0 installed.
## or anything more recent (when I write this 0.5.0 is the current
## development version)
##
## the examples below work in 0.5.0, some work in 0.4.0. 

import vispy
vispy.version_info

In [ ]:
# -*- coding: utf-8 -*-
# vispy: gallery 30
# -----------------------------------------------------------------------------
# Copyright (c) 2015, Vispy Development Team. All Rights Reserved.
# Distributed under the (new) BSD License. See LICENSE.txt for more info.
# -----------------------------------------------------------------------------
# Author: John David Reaver
# Date:   09/12/2014
# -----------------------------------------------------------------------------

"""
Example demonstrating the use of emulated double-precision floating point
numbers. Based off of mandelbrot.py.
The shader program emulates double-precision variables using a vec2 instead of
single-precision floats. Any function starting with ds_* operates on these
variables. See http://www.thasler.org/blog/?p=93.
NOTE: Some NVIDIA cards optimize the double-precision code away. Results are
therefore hardware dependent.
"""

from __future__ import division

import numpy as np
from vispy import app, gloo

# Shader source code
# -----------------------------------------------------------------------------
vertex = """
attribute vec2 position;
void main()
{
    gl_Position = vec4(position, 0, 1.0);
}
"""

fragment = """
#pragma optionNV(fastmath off)
#pragma optionNV(fastprecision off)
uniform vec2 inv_resolution_x;  // Inverse resolutions
uniform vec2 inv_resolution_y;
uniform vec2 center_x;
uniform vec2 center_y;
uniform vec2 scale;
uniform int iter;
// Jet color scheme
vec4 color_scheme(float x) {
    vec3 a, b;
    float c;
    if (x < 0.34) {
        a = vec3(0, 0, 0.5);
        b = vec3(0, 0.8, 0.95);
        c = (x - 0.0) / (0.34 - 0.0);
    } else if (x < 0.64) {
        a = vec3(0, 0.8, 0.95);
        b = vec3(0.85, 1, 0.04);
        c = (x - 0.34) / (0.64 - 0.34);
    } else if (x < 0.89) {
        a = vec3(0.85, 1, 0.04);
        b = vec3(0.96, 0.7, 0);
        c = (x - 0.64) / (0.89 - 0.64);
    } else {
        a = vec3(0.96, 0.7, 0);
        b = vec3(0.5, 0, 0);
        c = (x - 0.89) / (1.0 - 0.89);
    }
    return vec4(mix(a, b, c), 1.0);
}
vec2 ds_set(float a) {
    // Create an emulated double by storing first part of float in first half
    // of vec2
    vec2 z;
    z.x = a;
    z.y = 0.0;
    return z;
}
vec2 ds_add (vec2 dsa, vec2 dsb)
{
    // Add two emulated doubles. Complexity comes from carry-over.
    vec2 dsc;
    float t1, t2, e;
    t1 = dsa.x + dsb.x;
    e = t1 - dsa.x;
    t2 = ((dsb.x - e) + (dsa.x - (t1 - e))) + dsa.y + dsb.y;
    dsc.x = t1 + t2;
    dsc.y = t2 - (dsc.x - t1);
    return dsc;
}
vec2 ds_mul (vec2 dsa, vec2 dsb)
{
    vec2 dsc;
    float c11, c21, c2, e, t1, t2;
    float a1, a2, b1, b2, cona, conb, split = 8193.;
    cona = dsa.x * split;
    conb = dsb.x * split;
    a1 = cona - (cona - dsa.x);
    b1 = conb - (conb - dsb.x);
    a2 = dsa.x - a1;
    b2 = dsb.x - b1;
    c11 = dsa.x * dsb.x;
    c21 = a2 * b2 + (a2 * b1 + (a1 * b2 + (a1 * b1 - c11)));
    c2 = dsa.x * dsb.y + dsa.y * dsb.x;
    t1 = c11 + c2;
    e = t1 - c11;
    t2 = dsa.y * dsb.y + ((c2 - e) + (c11 - (t1 - e))) + c21;
    dsc.x = t1 + t2;
    dsc.y = t2 - (dsc.x - t1);
    return dsc;
}
// Compare: res = -1 if a < b
//              = 0 if a == b
//              = 1 if a > b
float ds_compare(vec2 dsa, vec2 dsb)
{
    if (dsa.x < dsb.x) return -1.;
    else if (dsa.x == dsb.x) {
        if (dsa.y < dsb.y) return -1.;
        else if (dsa.y == dsb.y) return 0.;
        else return 1.;
    }
    else return 1.;
}
void main() {
    vec2 z_x, z_y, c_x, c_y, x, y, frag_x, frag_y;
    vec2 four = ds_set(4.0);
    vec2 point5 = ds_set(0.5);
    // Recover coordinates from pixel coordinates
    frag_x = ds_set(gl_FragCoord.x);
    frag_y = ds_set(gl_FragCoord.y);
    c_x = ds_add(ds_mul(frag_x, inv_resolution_x), -point5);
    c_x = ds_add(ds_mul(c_x, scale), center_x);
    c_y = ds_add(ds_mul(frag_y, inv_resolution_y), -point5);
    c_y = ds_add(ds_mul(c_y, scale), center_y);
    // Main Mandelbrot computation
    int i;
    z_x = c_x;
    z_y = c_y;
    for(i = 0; i < iter; i++) {
        x = ds_add(ds_add(ds_mul(z_x, z_x), -ds_mul(z_y, z_y)), c_x);
        y = ds_add(ds_add(ds_mul(z_y, z_x), ds_mul(z_x, z_y)), c_y);
        if(ds_compare(ds_add(ds_mul(x, x), ds_mul(y, y)), four) > 0.) break;
        z_x = x;
        z_y = y;
    }
    // Convert iterations to color
    float color = 1.0 - float(i) / float(iter);
    gl_FragColor = color_scheme(color);
}
"""


# vispy Canvas
# -----------------------------------------------------------------------------
class Canvas(app.Canvas):

    def __init__(self, *args, **kwargs):
        app.Canvas.__init__(self, *args, **kwargs)
        self.program = gloo.Program(vertex, fragment)

        # Draw a rectangle that takes up the whole screen. All of the work is
        # done in the shader.
        self.program["position"] = [(-1, -1), (-1, 1), (1, 1),
                                    (-1, -1), (1, 1), (1, -1)]

        self.scale = 3
        self.program["scale"] = set_emulated_double(self.scale)
        self.center = [-0.5, 0]
        self.bounds = [-2, 2]
        self.translate_center(0, 0)

        self.iterations = self.program["iter"] = 300

        self.apply_zoom()

        self.min_scale = 1e-12
        self.max_scale = 4

        gloo.set_clear_color(color='black')

        self.show()

    def on_draw(self, event):
        self.program.draw()

    def on_resize(self, event):
        self.apply_zoom()

    def apply_zoom(self):
        width, height = self.physical_size
        gloo.set_viewport(0, 0, width, height)
        self.program['inv_resolution_x'] = set_emulated_double(1 / width)
        self.program['inv_resolution_y'] = set_emulated_double(1 / height)

    def on_mouse_move(self, event):
        """Pan the view based on the change in mouse position."""
        if event.is_dragging and event.buttons[0] == 1:
            x0, y0 = event.last_event.pos[0], event.last_event.pos[1]
            x1, y1 = event.pos[0], event.pos[1]
            X0, Y0 = self.pixel_to_coords(float(x0), float(y0))
            X1, Y1 = self.pixel_to_coords(float(x1), float(y1))
            self.translate_center(X1 - X0, Y1 - Y0)
            self.update()

    def translate_center(self, dx, dy):
        """Translates the center point, and keeps it in bounds."""
        center = self.center
        center[0] -= dx
        center[1] -= dy
        center[0] = min(max(center[0], self.bounds[0]), self.bounds[1])
        center[1] = min(max(center[1], self.bounds[0]), self.bounds[1])
        self.center = center

        center_x = set_emulated_double(center[0])
        center_y = set_emulated_double(center[1])
        self.program["center_x"] = center_x
        self.program["center_y"] = center_y

    def pixel_to_coords(self, x, y):
        """Convert pixel coordinates to Mandelbrot set coordinates."""
        rx, ry = self.size
        nx = (x / rx - 0.5) * self.scale + self.center[0]
        ny = ((ry - y) / ry - 0.5) * self.scale + self.center[1]
        return [nx, ny]

    def on_mouse_wheel(self, event):
        """Use the mouse wheel to zoom."""
        delta = event.delta[1]
        if delta > 0:  # Zoom in
            factor = 0.9
        elif delta < 0:  # Zoom out
            factor = 1 / 0.9
        for _ in range(int(abs(delta))):
            self.zoom(factor, event.pos)
        self.update()

    def on_key_press(self, event):
        """Use + or - to zoom in and out.
        The mouse wheel can be used to zoom, but some people don't have mouse
        wheels :)
        """
        if event.text == '+' or event.text == '=':
            self.zoom(0.9)
        elif event.text == '-':
            self.zoom(1/0.9)
        self.update()

    def zoom(self, factor, mouse_coords=None):
        """Factors less than zero zoom in, and greater than zero zoom out.
        If mouse_coords is given, the point under the mouse stays stationary
        while zooming. mouse_coords should come from MouseEvent.pos.
        """
        if mouse_coords is not None:  # Record the position of the mouse
            x, y = float(mouse_coords[0]), float(mouse_coords[1])
            x0, y0 = self.pixel_to_coords(x, y)

        self.scale *= factor
        self.scale = max(min(self.scale, self.max_scale), self.min_scale)
        self.program["scale"] = set_emulated_double(self.scale)

        if mouse_coords is not None:  # Translate so mouse point is stationary
            x1, y1 = self.pixel_to_coords(x, y)
            self.translate_center(x1 - x0, y1 - y0)


def set_emulated_double(number):
    """Emulate a double using two numbers of type float32."""
    double = np.array([number, 0], dtype=np.float32)  # Cast number to float32
    double[1] = number - double[0]  # Remainder stored in second half of array
    return double


if __name__ == '__main__':
    canvas = Canvas(size=(800, 800), keys='interactive')
    app.run()

## Other examples:

1) [Here](vispy.pqtorus.ipynb) is an example using vispy to generate and explore surfaces in $\mathbb R^3$.

2) [Here](plotly.pqtorus.ipynb) is essentially the same example, but using plotly, and putting the graphics on-line. 

3) Run python galaxy.py from the command line for another informative vispy demonstration. 

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# vispy: gallery 2, testskip
# Copyright (c) 2015, Vispy Development Team.
# Distributed under the (new) BSD License. See LICENSE.txt for more info.

"""
Shadertoy demo. You can copy-paste shader code from an example on
www.shadertoy.com and get the demo.
TODO: support cubes and videos as channel inputs (currently, only images
are supported).
"""

# NOTE: This example throws warnings about variables not being used;
# this is normal because only some shadertoy examples make use of all
# variables, and the GPU may compile some of them away.

import sys
from datetime import datetime, time
import numpy as np
from vispy import gloo
from vispy import app


vertex = """
#version 120
attribute vec2 position;
void main()
{
    gl_Position = vec4(position, 0.0, 1.0);
}
"""

fragment = """
#version 120
uniform vec3      iResolution;           // viewport resolution (in pixels)
uniform float     iGlobalTime;           // shader playback time (in seconds)
uniform vec4      iMouse;                // mouse pixel coords
uniform vec4      iDate;                 // (year, month, day, time in seconds)
uniform float     iSampleRate;           // sound sample rate (i.e., 44100)
uniform sampler2D iChannel0;             // input channel. XX = 2D/Cube
uniform sampler2D iChannel1;             // input channel. XX = 2D/Cube
uniform sampler2D iChannel2;             // input channel. XX = 2D/Cube
uniform sampler2D iChannel3;             // input channel. XX = 2D/Cube
uniform vec3      iChannelResolution[4]; // channel resolution (in pixels)
uniform float     iChannelTime[4];       // channel playback time (in sec)
%s
"""


def get_idate():
    now = datetime.now()
    utcnow = datetime.utcnow()
    midnight_utc = datetime.combine(utcnow.date(), time(0))
    delta = utcnow - midnight_utc
    return (now.year, now.month, now.day, delta.seconds)


def noise(resolution=64, nchannels=1):
    # Random texture.
    return np.random.randint(low=0, high=256,
                             size=(resolution, resolution, nchannels)
                             ).astype(np.uint8)


class Canvas(app.Canvas):

    def __init__(self, shadertoy=None):
        app.Canvas.__init__(self, keys='interactive')
        if shadertoy is None:
            shadertoy = """
            void main(void)
            {
                vec2 uv = gl_FragCoord.xy / iResolution.xy;
                gl_FragColor = vec4(uv,0.5+0.5*sin(iGlobalTime),1.0);
            }"""
        self.program = gloo.Program(vertex, fragment % shadertoy)

        self.program["position"] = [(-1, -1), (-1, 1), (1, 1),
                                    (-1, -1), (1, 1), (1, -1)]
        self.program['iMouse'] = 0, 0, 0, 0

        self.program['iSampleRate'] = 44100.
        for i in range(4):
            self.program['iChannelTime[%d]' % i] = 0.
        self.program['iGlobalTime'] = 0.

        self.activate_zoom()

        self._timer = app.Timer('auto', connect=self.on_timer, start=True)

        self.show()

    def set_channel_input(self, img, i=0):
        tex = gloo.Texture2D(img)
        tex.interpolation = 'linear'
        tex.wrapping = 'repeat'
        self.program['iChannel%d' % i] = tex
        self.program['iChannelResolution[%d]' % i] = img.shape

    def on_draw(self, event):
        self.program.draw()

    def on_mouse_click(self, event):
        # BUG: DOES NOT WORK YET, NO CLICK EVENT IN VISPY FOR NOW...
        imouse = event.pos + event.pos
        self.program['iMouse'] = imouse

    def on_mouse_move(self, event):
        if event.is_dragging:
            x, y = event.pos
            px, py = event.press_event.pos
            imouse = (x, self.size[1] - y, px, self.size[1] - py)
            self.program['iMouse'] = imouse

    def on_timer(self, event):
        self.program['iGlobalTime'] = event.elapsed
        self.program['iDate'] = get_idate()  # used in some shadertoy exs
        self.update()

    def on_resize(self, event):
        self.activate_zoom()

    def activate_zoom(self):
        gloo.set_viewport(0, 0, *self.physical_size)
        self.program['iResolution'] = (self.physical_size[0],
                                       self.physical_size[1], 0.)

# -------------------------------------------------------------------------
# COPY-PASTE SHADERTOY CODE BELOW
# -------------------------------------------------------------------------
SHADERTOY = """
// From: https://www.shadertoy.com/view/MdX3Rr
// Created by inigo quilez - iq/2013
// License Creative Commons Attribution-NonCommercial-ShareAlike 3.0
// Unported License.
//stereo thanks to Croqueteer
//#define STEREO
// value noise, and its analytical derivatives
vec3 noised( in vec2 x )
{
    vec2 p = floor(x);
    vec2 f = fract(x);
    vec2 u = f*f*(3.0-2.0*f);
    float a = texture2D(iChannel0,(p+vec2(0.5,0.5))/256.0,-100.0).x;
    float b = texture2D(iChannel0,(p+vec2(1.5,0.5))/256.0,-100.0).x;
    float c = texture2D(iChannel0,(p+vec2(0.5,1.5))/256.0,-100.0).x;
    float d = texture2D(iChannel0,(p+vec2(1.5,1.5))/256.0,-100.0).x;
    return vec3(a+(b-a)*u.x+(c-a)*u.y+(a-b-c+d)*u.x*u.y,
                6.0*f*(1.0-f)*(vec2(b-a,c-a)+(a-b-c+d)*u.yx));
}
const mat2 m2 = mat2(0.8,-0.6,0.6,0.8);
float terrain( in vec2 x )
{
    vec2  p = x*0.003;
    float a = 0.0;
    float b = 1.0;
    vec2  d = vec2(0.0);
    for( int i=0; i<6; i++ )
    {
        vec3 n = noised(p);
        d += n.yz;
        a += b*n.x/(1.0+dot(d,d));
        b *= 0.5;
        p = m2*p*2.0;
    }
    return 140.0*a;
}
float terrain2( in vec2 x )
{
    vec2  p = x*0.003;
    float a = 0.0;
    float b = 1.0;
    vec2  d = vec2(0.0);
    for( int i=0; i<14; i++ )
    {
        vec3 n = noised(p);
        d += n.yz;
        a += b*n.x/(1.0+dot(d,d));
        b *= 0.5;
        p=m2*p*2.0;
    }
    return 140.0*a;
}
float terrain3( in vec2 x )
{
    vec2  p = x*0.003;
    float a = 0.0;
    float b = 1.0;
    vec2  d = vec2(0.0);
    for( int i=0; i<4; i++ )
    {
        vec3 n = noised(p);
        d += n.yz;
        a += b*n.x/(1.0+dot(d,d));
        b *= 0.5;
        p = m2*p*2.0;
    }
    return 140.0*a;
}
float map( in vec3 p )
{
    float h = terrain(p.xz);
    return p.y - h;
}
float map2( in vec3 p )
{
    float h = terrain2(p.xz);
    return p.y - h;
}
float interesct( in vec3 ro, in vec3 rd )
{
    float h = 1.0;
    float t = 1.0;
    for( int i=0; i<120; i++ )
    {
        if( h<0.01 || t>2000.0 ) break;
        t += 0.5*h;
        h = map( ro + t*rd );
    }
    if( t>2000.0 ) t = -1.0;
    return t;
}
float sinteresct(in vec3 ro, in vec3 rd )
{
#if 0
    // no shadows
    return 1.0;
#endif
#if 0
    // fake shadows
    vec3 nor;
    vec3  eps = vec3(20.0,0.0,0.0);
    nor.x = terrain3(ro.xz-eps.xy) - terrain3(ro.xz+eps.xy);
    nor.y = 1.0*eps.x;
    nor.z = terrain3(ro.xz-eps.yx) - terrain3(ro.xz+eps.yx);
    nor = normalize(nor);
    return clamp( 4.0*dot(nor,rd), 0.0, 1.0 );
#endif
#if 1
    // real shadows
    float res = 1.0;
    float t = 0.0;
    for( int j=0; j<48; j++ )
    {
        vec3 p = ro + t*rd;
        float h = map( p );
        res = min( res, 16.0*h/t );
        t += h;
        if( res<0.001 ||p.y>300.0 ) break;
    }
    return clamp( res, 0.0, 1.0 );
#endif
}
vec3 calcNormal( in vec3 pos, float t )
{
    float e = 0.001;
    e = 0.001*t;
    vec3  eps = vec3(e,0.0,0.0);
    vec3 nor;
#if 0
    nor.x = map2(pos+eps.xyy) - map2(pos-eps.xyy);
    nor.y = map2(pos+eps.yxy) - map2(pos-eps.yxy);
    nor.z = map2(pos+eps.yyx) - map2(pos-eps.yyx);
#else
    nor.x = terrain2(pos.xz-eps.xy) - terrain2(pos.xz+eps.xy);
    nor.y = 2.0*e;
    nor.z = terrain2(pos.xz-eps.yx) - terrain2(pos.xz+eps.yx);
#endif
    return normalize(nor);
}
vec3 camPath( float time )
{
    vec2 p = 1100.0*vec2( cos(0.0+0.23*time), cos(1.5+0.21*time) );
    return vec3( p.x, 0.0, p.y );
}
float fbm( vec2 p )
{
    float f = 0.0;
    f += 0.5000*texture2D( iChannel0, p/256.0 ).x; p = m2*p*2.02;
    f += 0.2500*texture2D( iChannel0, p/256.0 ).x; p = m2*p*2.03;
    f += 0.1250*texture2D( iChannel0, p/256.0 ).x; p = m2*p*2.01;
    f += 0.0625*texture2D( iChannel0, p/256.0 ).x;
    return f/0.9375;
}
void main(void)
{
    vec2 xy = -1.0 + 2.0*gl_FragCoord.xy / iResolution.xy;
    vec2 s = xy*vec2(iResolution.x/iResolution.y,1.0);
    #ifdef STEREO
    float isCyan = mod(gl_FragCoord.x + mod(gl_FragCoord.y,2.0),2.0);
    #endif
    float time = iGlobalTime*0.15 + 0.3 + 4.0*iMouse.x/iResolution.x;
    vec3 light1 = normalize( vec3(-0.8,0.4,-0.3) );
    vec3 ro = camPath( time );
    vec3 ta = camPath( time + 3.0 );
    ro.y = terrain3( ro.xz ) + 11.0;
    ta.y = ro.y - 20.0;
    float cr = 0.2*cos(0.1*time);
    vec3  cw = normalize(ta-ro);
    vec3  cp = vec3(sin(cr), cos(cr),0.0);
    vec3  cu = normalize( cross(cw,cp) );
    vec3  cv = normalize( cross(cu,cw) );
    vec3  rd = normalize( s.x*cu + s.y*cv + 2.0*cw );
    #ifdef STEREO
    ro += 2.0*cu*isCyan;
    #endif
    float sundot = clamp(dot(rd,light1),0.0,1.0);
    vec3 col;
    float t = interesct( ro, rd );
    if( t<0.0 )
    {
        // sky
        col = vec3(0.3,.55,0.8)*(1.0-0.8*rd.y);
        col += 0.25*vec3(1.0,0.7,0.4)*pow( sundot,5.0 );
        col += 0.25*vec3(1.0,0.8,0.6)*pow( sundot,64.0 );
        col += 0.2*vec3(1.0,0.8,0.6)*pow( sundot,512.0 );
        vec2 sc = ro.xz + rd.xz*(1000.0-ro.y)/rd.y;
        col = mix( col, vec3(1.0,0.95,1.0),
            0.5*smoothstep(0.5,0.8,fbm(0.0005*sc)) );
    }
    else
    {
        // mountains
        vec3 pos = ro + t*rd;
        vec3 nor = calcNormal( pos, t );
        float r = texture2D( iChannel0, 7.0*pos.xz/256.0 ).x;
        col = (r*0.25+0.75)*0.9*mix( vec3(0.08,0.05,0.03),
            vec3(0.10,0.09,0.08), texture2D(iChannel0,0.00007*vec2(
                pos.x,pos.y*48.0)).x );
        col = mix( col, 0.20*vec3(0.45,.30,0.15)*(0.50+0.50*r),
            smoothstep(0.70,0.9,nor.y) );
        col = mix( col, 0.15*vec3(0.30,.30,0.10)*(0.25+0.75*r),
            smoothstep(0.95,1.0,nor.y) );
        // snow
        float h = smoothstep(55.0,80.0,pos.y + 25.0*fbm(0.01*pos.xz) );
        float e = smoothstep(1.0-0.5*h,1.0-0.1*h,nor.y);
        float o = 0.3 + 0.7*smoothstep(0.0,0.1,nor.x+h*h);
        float s = h*e*o;
        col = mix( col, 0.29*vec3(0.62,0.65,0.7), smoothstep(
            0.1, 0.9, s ) );
         // lighting
        float amb = clamp(0.5+0.5*nor.y,0.0,1.0);
        float dif = clamp( dot( light1, nor ), 0.0, 1.0 );
        float bac = clamp( 0.2 + 0.8*dot( normalize(
            vec3(-light1.x, 0.0, light1.z ) ), nor ), 0.0, 1.0 );
        float sh = 1.0; if( dif>=0.0001 ) sh = sinteresct(
            pos+light1*20.0,light1);
        vec3 lin  = vec3(0.0);
        lin += dif*vec3(7.00,5.00,3.00)*vec3( sh, sh*sh*0.5+0.5*sh,
            sh*sh*0.8+0.2*sh );
        lin += amb*vec3(0.40,0.60,0.80)*1.5;
        lin += bac*vec3(0.40,0.50,0.60);
        col *= lin;
        float fo = 1.0-exp(-0.0005*t);
        vec3 fco = 0.55*vec3(0.55,0.65,0.75) + 0.1*vec3(1.0,0.8,0.5)*pow(
            sundot, 4.0 );
        col = mix( col, fco, fo );
        col += 0.3*vec3(1.0,0.8,0.4)*pow( sundot,
                    8.0 )*(1.0-exp(-0.002*t));
    }
    col = pow(col,vec3(0.4545));
    // vignetting
    col *= 0.5 + 0.5*pow( (xy.x+1.0)*(xy.y+1.0)*(xy.x-1.0)*(xy.y-1.0),
                         0.1 );
    #ifdef STEREO
    col *= vec3( isCyan, 1.0-isCyan, 1.0-isCyan );
    #endif
//	col *= smoothstep( 0.0, 2.0, iGlobalTime );
    gl_FragColor=vec4(col,1.0);
}
"""
# -------------------------------------------------------------------------

canvas = Canvas(SHADERTOY)
# Input data.
canvas.set_channel_input(noise(resolution=256, nchannels=1), i=0)

if __name__ == '__main__':

    canvas.show()
    if sys.flags.interactive == 0:
        canvas.app.run()

See the [Vispy Gallery](http://vispy.org/gallery.html) for more examples of what Vispy can do.  Also see the [table of examples](https://github.com/vispy/vispy/tree/master/examples).  A few random choices appear below. 

In [ ]:
from vispy import app, gloo

vertex = """
attribute vec2 a_position;
varying vec2 v_position;
void main()
{
    gl_Position = vec4(a_position, 0.0, 1.0);
    v_position = a_position;
}
"""

fragment = """
#include "math/constants.glsl"
//const float M_PI = 3.14159265358979323846;
uniform float u_time;
varying vec2 v_position;
/**********************************************************
Specify the parameters here.
**********************************************************/
const float z_offset = 1.;  // (z+z_offset)/z_max should be in [0,1]
const float z_max = 2.;
const float x_scale = 5.;  // x is between -x_scale and +x_scale
const float y_scale = 5.; // y is between -y_scale and +y_scale
const float t_scale = 5.; // scale for the time
/*********************************************************/
float f(float x, float y, float t) {
    // x is in [-x_scale, +x_scale]
    // y is in [-y_scale, +y_scale]
    // t is in [0, +oo)
    /**********************************************************
    Write your function below.
    **********************************************************/
    float k = .25*cos(t);
    return (cos(x)+k)*(sin(y)-k);
    /*********************************************************/
}
vec4 jet(float x) {
    vec3 a, b;
    float c;
    if (x < 0.34) {
        a = vec3(0, 0, 0.5);
        b = vec3(0, 0.8, 0.95);
        c = (x - 0.0) / (0.34 - 0.0);
    } else if (x < 0.64) {
        a = vec3(0, 0.8, 0.95);
        b = vec3(0.85, 1, 0.04);
        c = (x - 0.34) / (0.64 - 0.34);
    } else if (x < 0.89) {
        a = vec3(0.85, 1, 0.04);
        b = vec3(0.96, 0.7, 0);
        c = (x - 0.64) / (0.89 - 0.64);
    } else {
        a = vec3(0.96, 0.7, 0);
        b = vec3(0.5, 0, 0);
        c = (x - 0.89) / (1.0 - 0.89);
    }
    return vec4(mix(a, b, c), 1.0);
}
void main() {
    vec2 pos = v_position;
    float z = f(x_scale * pos.x, y_scale * pos.y, t_scale * u_time);
    gl_FragColor = jet((z + z_offset) / (z_max));
}
"""


class Canvas(app.Canvas):
    def __init__(self):
        app.Canvas.__init__(self, position=(300, 100),
                            size=(800, 800), keys='interactive')

        self.program = gloo.Program(vertex, fragment)
        self.program['a_position'] = [(-1., -1.), (-1., +1.),
                                      (+1., -1.), (+1., +1.)]

        self.program['u_time'] = 0.0
        self.timer = app.Timer('auto', connect=self.on_timer, start=True)

        self.show()

    def on_timer(self, event):
        self.program['u_time'] = event.elapsed
        self.update()

    def on_resize(self, event):
        width, height = event.physical_size
        gloo.set_viewport(0, 0, width, height)

    def on_draw(self, event):
        self.program.draw('triangle_strip')

if __name__ == '__main__':
    canvas = Canvas()
    app.run()
    

In [ ]:


""" Demonstrates use of visual.Markers to create a point cloud with a
standard turntable camera to fly around with and a centered 3D Axis.
"""

import numpy as np
import vispy.scene
from vispy.scene import visuals

#
# Make a canvas and add simple view
#
canvas = vispy.scene.SceneCanvas(keys='interactive', show=True)
view = canvas.central_widget.add_view()


# generate data
pos = np.random.normal(size=(200000, 3), scale=0.2)
# one could stop here for the data generation, the rest is just to make the
# data look more interesting. Copied over from magnify.py
centers = np.random.normal(size=(50, 3))
indexes = np.random.normal(size=200000, loc=centers.shape[0]/2.,
                           scale=centers.shape[0]/3.)
indexes = np.clip(indexes, 0, centers.shape[0]-1).astype(int)
scales = 10**(np.linspace(-2, 0.5, centers.shape[0]))[indexes][:, np.newaxis]
pos *= scales
pos += centers[indexes]

# create scatter object and fill in the data
scatter = visuals.Markers()
scatter.set_data(pos, edge_color=None, face_color=(1, 1, 1, .5), size=5)

view.add(scatter)

view.camera = 'turntable'  # or try 'arcball'

# add a colored 3D axis for orientation
axis = visuals.XYZAxis(parent=view.scene)

if __name__ == '__main__':
    import sys
    if sys.flags.interactive != 1:
        vispy.app.run()

In [ ]:
"""
This example demonstrates the use of the Isosurface visual.
"""

import sys
import numpy as np

from vispy import app, scene

# Create a canvas with a 3D viewport
canvas = scene.SceneCanvas(keys='interactive')
view = canvas.central_widget.add_view()

res=50

## Define a scalar field from which we will generate an isosurface
def psi(i, j, k, offset=(res, res, 2*res)):
    x = i-offset[0]
    y = j-offset[1]
    z = k-offset[2]
    th = np.arctan2(z, (x**2+y**2)**0.5)
    r = (x**2 + y**2 + z**2)**0.5
    a0 = 1
    ps = ((1./81.) * 1./(6.*np.pi)**0.5 * (1./a0)**(3/2) * (r/a0)**2 *
          np.exp(-r/(3*a0)) * (3 * np.cos(th)**2 - 1))
    return ps

print("Generating scalar field..")
data = np.abs(np.fromfunction(psi, (2*res, 2*res, 4*res)))

# Create isosurface visual
surface = scene.visuals.Isosurface(data, level=data.max()/4.,
                                   color=(0.5, 0.6, 1, 1), shading='smooth',
                                   parent=view.scene)
surface.transform = scene.transforms.STTransform(translate=(-res, -res, -2*res))

# Add a 3D axis to keep us oriented
axis = scene.visuals.XYZAxis(parent=view.scene)

# Use a 3D camera
# Manual bounds; Mesh visual does not provide bounds yet
# Note how you can set bounds before assigning the camera to the viewbox
cam = scene.TurntableCamera(elevation=30, azimuth=30)
cam.set_range((-10, 10), (-10, 10), (-10, 10))
view.camera = cam


if __name__ == '__main__':
    canvas.show()
    if sys.flags.interactive == 0:
        app.run()
        

In [ ]:
"""
Simple use of SceneCanvas to display a cube with an arcball camera.
"""
import sys

from vispy import scene
from vispy.color import Color
canvas = scene.SceneCanvas(keys='interactive', size=(800, 600), show=True)

# Set up a viewbox to display the cube with interactive arcball
view = canvas.central_widget.add_view()
view.bgcolor = '#efefef'
view.camera = 'turntable'
view.padding = 100

color = Color("#3f51b5")

cube = scene.visuals.Cube(size=1, color=color, edge_color="black",
                          parent=view.scene)
if __name__ == '__main__' and sys.flags.interactive == 0:
    canvas.app.run()
    

In [ ]:
"""
Show vector field flow
"""

from __future__ import division

from vispy import app, scene, visuals, gloo
from vispy.util import ptime
import numpy as np


class VectorFieldVisual(visuals.Visual):
    vertex = """
    uniform sampler2D field;
    attribute vec2 index;
    uniform vec2 shape;
    uniform vec2 field_shape;
    uniform float spacing;
    varying float dist;  // distance along path for this vertex
    varying vec2 ij;
    uniform sampler2D offset;
    uniform float seg_len;
    uniform int n_iter;  // iterations to integrate along field per vertex
    uniform vec2 attractor;
    varying vec4 base_color;
    uniform sampler2D color;
    
    void main() {
        // distance along one line
        dist = index.y * seg_len;
        
        vec2 local;
        ij = vec2(mod(index.x, shape.x), floor(index.x / shape.x));
        // *off* is a random offset to the starting location, which prevents
        // the appearance of combs in the field 
        vec2 off = texture2D(offset, ij / shape).xy - 0.5;
        local = spacing * (ij + off);
        vec2 uv;
        vec2 dir;
        vec2 da;
        for( int i=0; i<index.y; i+=1 ) {
            for ( int j=0; j<n_iter; j += 1 ) {
                uv = local / field_shape;
                dir = texture2D(field, uv).xy;
                
                // add influence of variable attractor (mouse)
                da = attractor - local;
                float al = 0.1 * length(da);
                da /= 0.5 * (1 + al*al);
                
                dir += da;
                
                // maybe pick a more accurate integration method?
                local += seg_len * dir / n_iter;
            }
        }
        base_color = texture2D(color, uv);
        
        gl_Position = $transform(vec4(local, 0, 1));
    }
    """
    
    fragment = """
    uniform float time;
    uniform float speed;
    varying float dist;
    varying vec2 ij;
    uniform sampler2D offset;
    uniform vec2 shape;
    uniform float nseg;
    uniform float seg_len;
    varying vec4 base_color;
    
    void main() {
        float totlen = nseg * seg_len;
        float phase = texture2D(offset, ij / shape).b;
        float alpha;
        
        // vary alpha along the length of the line to give the appearance of
        // motion
        alpha = mod((dist / totlen) + phase - time * speed, 1);
        
        // add a cosine envelope to fade in and out smoothly at the ends
        alpha *= (1 - cos(2 * 3.141592 * dist / totlen)) * 0.5;
        
        gl_FragColor = vec4(base_color.rgb, base_color.a * alpha);
    }
    """
    
    def __init__(self, field, spacing=10, segments=3, seg_len=0.5,
                 color=(1, 1, 1, 0.3)):
        self._time = 0.0
        self._last_time = ptime.time()
        rows = field.shape[0] / spacing
        cols = field.shape[1] / spacing
        index = np.empty((rows * cols, segments * 2, 2), dtype=np.float32)
        
        # encodes starting position within vector field
        index[:, :, 0] = np.arange(rows * cols)[:, np.newaxis]
        # encodes distance along length of line
        index[:, ::2, 1] = np.arange(segments)[np.newaxis, :]
        index[:, 1::2, 1] = np.arange(segments)[np.newaxis, :] + 1
        self._index = gloo.VertexBuffer(index)
        if not isinstance(color, np.ndarray):
            color = np.array([[list(color)]], dtype='float32')
        self._color = gloo.Texture2D(color)
        
        offset = np.random.uniform(256, size=(rows, cols, 3)).astype(np.ubyte)
        self._offset = gloo.Texture2D(offset, format='rgb')
        self._field = gloo.Texture2D(field, format='rg',
                                     internalformat='rg32f',
                                     interpolation='linear')
        self._field_shape = field.shape[:2]
        
        visuals.Visual.__init__(self, vcode=self.vertex, fcode=self.fragment)
        self.timer = app.Timer(interval='auto', connect=self.update_time,
                               start=False)
        self.freeze()
        
        self.shared_program['field'] = self._field
        self.shared_program['field_shape'] = self._field.shape[:2]
        self.shared_program['shape'] = (rows, cols)
        self.shared_program['index'] = self._index
        self.shared_program['spacing'] = spacing
        self.shared_program['t'] = self._time
        self.shared_program['offset'] = self._offset
        self.shared_program['speed'] = 1
        self.shared_program['color'] = self._color
        self.shared_program['seg_len'] = seg_len
        self.shared_program['nseg'] = segments
        self.shared_program['n_iter'] = 1
        self.shared_program['attractor'] = (0, 0)
        self.shared_program['time'] = 0
        self._draw_mode = 'lines'
        self.set_gl_state('translucent', depth_test=False)
        
        self.timer.start()
        
    def _prepare_transforms(self, view):
        view.view_program.vert['transform'] = view.get_transform()
        
    def _prepare_draw(self, view):
        pass
    
    def _compute_bounds(self, axis, view):
        if axis > 1:
            return (0, 0)
        return (0, self._field_shape[axis])

    def update_time(self, ev):
        t = ptime.time()
        self._time += t - self._last_time
        self._last_time = t
        self.shared_program['time'] = self._time
        self.update()


VectorField = scene.visuals.create_visual_node(VectorFieldVisual)


def fn(y, x):
    dx = x-50
    dy = y-30
    l = (dx**2 + dy**2)**0.5 + 0.01
    return np.array([100 * dy / l**1.7, -100 * dx / l**1.8])

field = np.fromfunction(fn, (100, 100)).transpose(1, 2, 0).astype('float32')
field[..., 0] += 10 * np.cos(np.linspace(0, 2 * 3.1415, 100))

color = np.zeros((100, 100, 4), dtype='float32')
color[..., :2] = (field + 5) / 10.
color[..., 2] = 0.5
color[..., 3] = 0.5

canvas = scene.SceneCanvas(keys='interactive', show=True)
view = canvas.central_widget.add_view(camera='panzoom')

vfield = VectorField(field[..., :2], spacing=0.5, segments=30, seg_len=0.05,
                     parent=view.scene, color=color)
view.camera.set_range()


@canvas.connect
def on_mouse_move(event):
    if 3 in event.buttons:
        tr = canvas.scene.node_transform(vfield)
        vfield.shared_program['attractor'] = tr.map(event.pos)[:2]


if __name__ == '__main__':
    app.run()
    

In [ ]:
"""
Plot clusters of data points and a graph of connections
"""
from vispy import app, scene, color
import numpy as np

# Initialize arrays for position, color, edges, and types for each point in
# the graph.
npts = 400
nedges = 900
ngroups = 7
np.random.seed(127396)
pos = np.empty((npts, 2), dtype='float32')
colors = np.empty((npts, 3), dtype='float32')
edges = np.empty((nedges, 2), dtype='uint32')
types = np.empty(npts, dtype=int)

# Assign random starting positions
pos[:] = np.random.normal(size=pos.shape, scale=4.)

# Assign each point to a group
grpsize = npts // ngroups
ptr = 0
typ = 0
while ptr < npts:
    size = np.random.random() * grpsize + grpsize // 2
    types[ptr:ptr+size] = typ
    typ += 1
    ptr = ptr + size

# Randomly select connections, with higher connection probability between 
# points in the same group
conn = []
connset = set()
while len(conn) < nedges:
    i, j = np.random.randint(npts, size=2)
    if i == j:
        continue
    p = 0.7 if types[i] == types[j] else 0.01
    if np.random.random() < p:
        if (i, j) in connset:
            continue
        connset.add((i, j))
        connset.add((j, i))
        conn.append([i, j])
edges[:] = conn

# Assign colors to each point based on its type
cmap = color.get_colormap('cubehelix')
typ_colors = np.array([cmap.map(x)[0, :3] for x in np.linspace(0.2, 0.8, typ)])
colors[:] = typ_colors[types]

# Add some RGB noise and clip
colors *= 1.1 ** np.random.normal(size=colors.shape)
colors = np.clip(colors, 0, 1)


# Display the data
canvas = scene.SceneCanvas(keys='interactive', show=True)
view = canvas.central_widget.add_view()
view.camera = 'panzoom'
view.camera.aspect = 1

lines = scene.Line(pos=pos, connect=edges, antialias=False, method='gl',
                   color=(1, 1, 1, 0.2), parent=view.scene)
markers = scene.Markers(pos=pos, face_color=colors, symbol='o',
                        parent=view.scene)

view.camera.set_range()

i = 1


def update(ev):
    global pos, edges, lines, markers, view, force, dist, i
    
    dx = np.empty((npts, npts, 2), dtype='float32')
    dx[:] = pos[:, np.newaxis, :]
    dx -= pos[np.newaxis, :, :]

    dist = (dx**2).sum(axis=2)**0.5
    dist[dist == 0] = 1.
    ndx = dx / dist[..., np.newaxis]
    
    force = np.zeros((npts, npts, 2), dtype='float32')
    
    # all points push away from each other
    force -= 0.1 * ndx / dist[..., np.newaxis]**2
    
    # connected points pull toward each other
    # pulsed force helps to settle faster:    
    s = 0.1
    #s = 0.05 * 5 ** (np.sin(i/20.) / (i/100.))
    
    #s = 0.05 + 1 * 0.99 ** i
    mask = np.zeros((npts, npts, 1), dtype='float32')
    mask[edges[:, 0], edges[:, 1]] = s
    mask[edges[:, 1], edges[:, 0]] = s
    force += dx * dist[..., np.newaxis] * mask
    
    # points do not exert force on themselves
    force[np.arange(npts), np.arange(npts)] = 0
    
    force = force.sum(axis=0)
    pos += np.clip(force, -3, 3) * 0.09
    
    lines.set_data(pos=pos)
    markers.set_data(pos=pos, face_color=colors)
    
    i += 1


timer = app.Timer(interval=0, connect=update, start=True)

if __name__ == '__main__':
    app.run()
    

In [ ]:

"""
Multiple real-time digital signals with GLSL-based clipping.
"""

from vispy import gloo
from vispy import app
import numpy as np
import math

# Number of cols and rows in the table.
nrows = 80
ncols = 80

# Number of signals.
m = nrows*ncols

# Number of samples per signal.
n = 1000

# Various signal amplitudes.
amplitudes = .1 + .2 * np.random.rand(m, 1).astype(np.float32)

# Generate the signals as a (m, n) array.
y = amplitudes * np.random.randn(m, n).astype(np.float32)

# Color of each vertex (TODO: make it more efficient by using a GLSL-based
# color map and the index).
color = np.repeat(np.random.uniform(size=(m, 3), low=.5, high=.9),
                  n, axis=0).astype(np.float32)

# Signal 2D index of each vertex (row and col) and x-index (sample index
# within each signal).
index = np.c_[np.repeat(np.repeat(np.arange(ncols), nrows), n),
              np.repeat(np.tile(np.arange(nrows), ncols), n),
              np.tile(np.arange(n), m)].astype(np.float32)

VERT_SHADER = """
#version 120
// y coordinate of the position.
attribute float a_position;
// row, col, and time index.
attribute vec3 a_index;
varying vec3 v_index;
// 2D scaling factor (zooming).
uniform vec2 u_scale;
// Size of the table.
uniform vec2 u_size;
// Number of samples per signal.
uniform float u_n;
// Color.
attribute vec3 a_color;
varying vec4 v_color;
// Varying variables used for clipping in the fragment shader.
varying vec2 v_position;
varying vec4 v_ab;
void main() {
    float nrows = u_size.x;
    float ncols = u_size.y;
    // Compute the x coordinate from the time index.
    float x = -1 + 2*a_index.z / (u_n-1);
    vec2 position = vec2(x - (1 - 1 / u_scale.x), a_position);
    // Find the affine transformation for the subplots.
    vec2 a = vec2(1./ncols, 1./nrows)*.9;
    vec2 b = vec2(-1 + 2*(a_index.x+.5) / ncols,
                  -1 + 2*(a_index.y+.5) / nrows);
    // Apply the static subplot transformation + scaling.
    gl_Position = vec4(a*u_scale*position+b, 0.0, 1.0);
    v_color = vec4(a_color, 1.);
    v_index = a_index;
    // For clipping test in the fragment shader.
    v_position = gl_Position.xy;
    v_ab = vec4(a, b);
}
"""

FRAG_SHADER = """
#version 120
varying vec4 v_color;
varying vec3 v_index;
varying vec2 v_position;
varying vec4 v_ab;
void main() {
    gl_FragColor = v_color;
    // Discard the fragments between the signals (emulate glMultiDrawArrays).
    if ((fract(v_index.x) > 0.) || (fract(v_index.y) > 0.))
        discard;
    // Clipping test.
    vec2 test = abs((v_position.xy-v_ab.zw)/v_ab.xy);
    if ((test.x > 1) || (test.y > 1))
        discard;
}
"""


class Canvas(app.Canvas):
    def __init__(self):
        app.Canvas.__init__(self, title='Use your wheel to zoom!',
                            keys='interactive')
        self.program = gloo.Program(VERT_SHADER, FRAG_SHADER)
        self.program['a_position'] = y.reshape(-1, 1)
        self.program['a_color'] = color
        self.program['a_index'] = index
        self.program['u_scale'] = (1., 1.)
        self.program['u_size'] = (nrows, ncols)
        self.program['u_n'] = n

        gloo.set_viewport(0, 0, *self.physical_size)

        self._timer = app.Timer('auto', connect=self.on_timer, start=True)

        gloo.set_state(clear_color='black', blend=True,
                       blend_func=('src_alpha', 'one_minus_src_alpha'))

        self.show()

    def on_resize(self, event):
        gloo.set_viewport(0, 0, *event.physical_size)

    def on_mouse_wheel(self, event):
        dx = np.sign(event.delta[1]) * .05
        scale_x, scale_y = self.program['u_scale']
        scale_x_new, scale_y_new = (scale_x * math.exp(2.5*dx),
                                    scale_y * math.exp(0.0*dx))
        self.program['u_scale'] = (max(1, scale_x_new), max(1, scale_y_new))
        self.update()

    def on_timer(self, event):
        """Add some data at the end of each signal (real-time signals)."""
        k = 10
        y[:, :-k] = y[:, k:]
        y[:, -k:] = amplitudes * np.random.randn(m, k)

        self.program['a_position'].set_data(y.ravel().astype(np.float32))
        self.update()

    def on_draw(self, event):
        gloo.clear()
        self.program.draw('line_strip')

if __name__ == '__main__':
    c = Canvas()
    app.run()